# Named Entity Recognition

## Natural Language Processing

### Overview

In this walkthrough, we will be using a Hidden Markov Model to perform Named Entity Recognition to gain experience with sequence labeling.

**Please note: The data that we will be using is relatively unfiltered, real world data that comes from tweets. It contains some vulgar language that has not been filtered out.**

### Setup

The training data in `data/train` is in the following format:

    Greek B-other
    Festival I-other
    at O
    St B-facility
    Johns I-facility
    before O
    ASPEN B-geo-loc
    
    Ew O
    wait O
    ...

It is organized with two whitespace-seperated columns as seen above. The first column contains the word and the second column contains that word's corresponding BIO tag. Blank lines seperate one sentence from another.

The code below reads in the training data and stores it as a list called `train` in the following format:

    [
    ["Greek Festival at St Johns before ASPEN", ["B-other", "I-other", "O", "B-facility", "I-facility", "O", "B-geo-loc"]],
    ["Ew wait ...", ["O", "O", ...]],
    ...
    ]

Each element in the list should be a list of length 2. The first element of this list should be the sentence as a space-seperated string and the second element in this list should be a list of the associated tags.

In [1]:
train = [' '.join(i.split('\n')).replace('\t', ' ').split() for i in open('data/train', 'r').read().split('\n\n')]
train = [[" ".join(i[::2]),i[1::2]] for i in train]

First, we create a list called `tags` containing all of the unique tags found in the training data and a list called `words` containing all of the unique words found in the training data.

In [2]:
### ENTER CODE HERE ###

In [3]:
### ENTER CODE HERE ###

# Print the results
print(f"Tokens: {sum([len(i[1]) for i in train])}, Word Types: {len(words)}, Tag Types: {len(tags)} ")

Tokens: 46235, Word Types: 10553, Tag Types: 21 


### Hidden Markov Model

Recall that a Hidden Markov Model (HMM) is a type of statistical model that's used for sequential data. HMMs assume that an underlying process is governed by a Markov process with unobservable (i.e., hidden) states. The model is called "hidden" because the states and state transitions aren't directly observable; instead, we observe some output that is generated from each state.

The Markov assumption means that the probability of transitioning to any particular state depends solely on the current state and time, independent of the sequence of states that preceded it. This sequence property makes Markov models particularly useful for algorithms like Viterbi, which finds the most likely sequence of hidden states given the sequence of observations.

We will use the HMM in this task for part-of-speech (POS) tagging. In POS tagging, the goal is to label each word in a sentence with its appropriate part of speech, such as noun, verb, adjective, etc.

First, we will add the following words to the `words` list:
- `<B>` to denote the beginning of a sentence
- `<E>` to denote the end of a sentence
- `<UNK>` for any unseen words

And the following tags to the `tags` list:
- `<B>` to tag the `<B>` word
- `<E>` to tag the `<E>` word

In [4]:
### ENTER CODE HERE ###

Now, we will create an `HMM` class with the following functions:

- **'train'** function: This function takes one 'train data' set, creates one list of "tags", one list of "words" and counts each tag, each tag-tag pair, and each word-tag pair occurance in the training data. 
 
- **'transition_proba'** function: That is the 'Transition Probabilities' Function. It takes on **tag-tag** pair and returns the relative frequency estimation for this given **tag-tag** pair. Recall that the relative frequency estimation for an HMM involves both **transition probabilities** (tag1-tag2 pair count / tag1 count) and **emission probabilities** (word-tag pair counts / tag count), but for this part we will only implement the 'Transition Probabilities'.

In [5]:
class HMM():
    
    def __init__(self):
        self.words=set()
        self.tags=set()
        
        self.tag_counts = {}
        self.tagtag_counts = {}
        self.wordtag_counts = {}
        
    def train(self, train, words, tags):
        import itertools
        self.tags = tags
        self.words = words

        self.tag_counts          = {(i): 0 for i in self.tags}      
        self.tagtag_counts       = {(i[0], i[1]): 0 for i in itertools.product(self.tags, self.tags)}
        self.wordtag_counts      = {(i[0], i[1]): 0 for i in itertools.product(self.words, self.tags)}
        
        
        for sentence_tag in train: #for each sentence
            xtags = sentence_tag[1].copy()
            xtags.insert(0,'<B>')
            xtags.append('<E>')


            xsentence = sentence_tag[0].split()
            xsentence.insert(0,'<B>')
            xsentence.append('<E>')
            
            #count tags
            ### ENTER CODE HERE ###    
  
            #count tags-tags
            ### ENTER CODE HERE ###

            #count word-tag       
            ### ENTER CODE HERE ###
         
            
    def transition_proba(self, tag_tag): #transitions (tag1-tag2 pair count / tag1 count)  
        ### ENTER CODE HERE ###

In [6]:
### ENTER CODE HERE ###

In [7]:
print("p(B-person | O) = " + str(hmm.transition_proba(('O', 'B-person'))))
print("p(B-person | B-person) = " + str(hmm.transition_proba(('B-person', 'B-person'))))
print("p(I-person | B-person) = " + str(hmm.transition_proba(('B-person', 'I-person'))))
print("p(B-person | I-person) = " + str(hmm.transition_proba(('I-person', 'B-person'))))
print("p(I-person | I-person) = " + str(hmm.transition_proba(('I-person', 'I-person'))))
print("p(O | I-person) = " + str(hmm.transition_proba(('I-person', 'O'))))

p(B-person | O) = 0.009114583333333334
p(B-person | B-person) = 0.0
p(I-person | B-person) = 0.4375
p(B-person | I-person) = 0.0
p(I-person | I-person) = 0.08411214953271028
p(O | I-person) = 0.8878504672897196


### Smoothing

In the context of Hidden Markov Models (HMMs), "smoothing" refers to a method used to estimate the "hidden" state of a system at a particular point in time, given all of the observed data (i.e., not just data up to that point, but data that comes after it as well). It's called "smoothing" because it uses future data to "smooth" out the prediction for the current state.

We will copy our HMM class from above and modify it by smoothing the emission probabilities. This involves the following: 

- Creating a new dictionary called `self.smoothed_tag_counts` for the smoothed tag counts. In the Train Function, we will use it to count the Tags and add 0.1 to each 'word' of the 'word-tag' counts.
- Initializing the wordtag_counts with 0.1 instead of 0
- Creating an Emission Probabilities function called `emission_proba` that returns the relative frequency estimation for a given 'word-tag' pair. In this function, if the 'word' from the 'word-tag' is unknown (not seen before in the training), we will return the emission probability of "\<UNK\>"-tag instead.

In [8]:
class smoothHMM():
    
    def __init__(self):
        self.words=set()
        self.tags=set()
        
        self.tag_counts = {}
        self.tagtag_counts = {}
        self.wordtag_counts = {}
        self.smoothed_tag_counts = {}
        
    def train(self, train, words, tags):
        import itertools
        self.tags = tags
        self.words = words

        self.tag_counts          = {(i): 0 for i in self.tags}      
        self.smoothed_tag_counts = {(i): 0 for i in self.tags} 
        self.tagtag_counts       = {(i[0], i[1]): 0 for i in itertools.product(self.tags, self.tags)}
        self.wordtag_counts      = {(i[0], i[1]): 0.1 for i in itertools.product(self.words, self.tags)}
        
        ### ENTER CODE HERE ###
        
        for sentence_tag in train: #for each sentence
            xtags = sentence_tag[1].copy()
            xtags.insert(0,'<B>')
            xtags.append('<E>')


            xsentence = sentence_tag[0].split()
            xsentence.insert(0,'<B>')
            xsentence.append('<E>')
            
            #count tags
            for xtag in xtags:
                self.tag_counts[xtag]+=1      
                ### ENTER CODE HERE ###

            #count tags-tags
            for i in range(len(xtags)-1):
                self.tagtag_counts[(xtags[i],xtags[i+1])]+=1  

            #count word-tag       
            for i in range(len(xtags)-1):
                self.wordtag_counts[(xsentence[i],xtags[i])]+=1   

            
    def transition_proba(self, tag_tag): #transitions (tag1-tag2 pair count / tag1 count)  
        tag_tag_count = self.tagtag_counts[tag_tag]
        tag1_count = self.tag_counts[tag_tag[0]]
        return tag_tag_count / tag1_count
        
    def emission_proba(self, word_tag): #emissions (word-tag pair counts / tag count).
        ### ENTER CODE HERE ###

In [9]:
### ENTER CODE HERE ###
print(f"p(God | B-person) {hmm.emission_proba(('God','B-person'))}")
print(f"p(God | O) {hmm.emission_proba(('God','O'))}")
print(f"p(Lindsay | B-person) {hmm.emission_proba(('Lindsay','B-person'))}")
print(f"p(Lindsay | O) {hmm.emission_proba(('Lindsay','O'))}")

p(God | B-person) 0.006052141527001317
p(God | O) 0.000136064740049429
p(Lindsay | B-person) 0.008047353019419334
p(Lindsay | O) 2.230569509007033e-06


### Viterbi Algorithm

The Viterbi algorithm is a dynamic programming algorithm used for finding the most likely sequence of hidden states in a hidden Markov model (HMM). It is widely used in various fields, including natural language processing (NLP), speech recognition, bioinformatics, and more.

In NLP, the Viterbi algorithm is particularly important for part-of-speech tagging and named entity recognition tasks. These tasks involve determining the most likely sequence of grammatical categories (part-of-speech tags) or named entities (such as names of people, organizations, locations, etc.) that correspond to a given sequence of words in a sentence.

We will create a Viterbi Function called `viterbi` that implements the viterbi algorithm. The function will take a list with a sentence String as the first element in the list and a list of tags as the second element in the list. 

It will return the sequence of BIO tags that gives the maximum score for that sentence as a list.

#### Example

**Input:**

    ["STOP WHAT YOU'RE DOING AND", ['B-other', 'I-other', 'I-other', 'I-other', 'I-other']]
    
    
**Output**

    ['B-other', 'I-other', 'I-other', 'I-other', 'O']

In [10]:
def viterbi(hmm, sent, printIt=''):
    import itertools
    import pprint
    pp = pprint.PrettyPrinter(indent=4)
    
    sentence = sent[0].split()   
    
    tags = [i for i in hmm.tags if i not in ['<B>', '<E>']]    

    table = []
    for w in range(len(sentence)):        
        word = sentence[w]        
        for tag in tags:
            #create structre and get emission value
            
            entry = {"prev_key"      :('' ,'', 0),                
                     'prev_value'    :0, 
                     "emission"      :hmm.emission_proba((word, tag)),
                     "max_transition":0, 
                     'value'         :0}
            
            #get the max transition value
            if w ==0:
                entry['max_transition'] = hmm.transition_proba(('<B>', tag))
                entry['prev_key']       = ('<B>', '<B>', -1)
                entry['prev_value']     = 0
                entry['value']          = entry['max_transition'] * entry['emission'] 

            else:
                transition_max = float("-inf")
                max_value = float("-inf")
                best_prev_entry = None

                for prev_entry in [d for d in table if list(d.keys())[0][2] == w-1]: #all entries from previous levels
                    prev_entry_tag = list(prev_entry.keys())[0][1]                    
                    prev_entry_value = prev_entry[list(prev_entry.keys())[0]]["value"]

                    transition = hmm.transition_proba((prev_entry_tag, tag))
                    xvalue = entry["emission"] * transition * prev_entry_value
                    
                    
                    if xvalue >= max_value:
                        transition_max = transition
                        max_value = xvalue
                        best_prev_entry = prev_entry

                entry['max_transition'] = transition_max
                entry['prev_key'] = list(best_prev_entry.keys())[0]
                entry['prev_value'] = best_prev_entry[list(best_prev_entry.keys())[0]]["value"]
                entry['value'] = entry['max_transition'] * entry['emission'] * entry['prev_value']
            
            table.append({(word, tag, w): entry})
            
    if printIt=='all': 
        for i in table:
            pp.pprint(i)
            print("\n")
        print("\n")
    
    
    #Backtrack
    BIO_list = []
    final_table = []
    
    #get the last level of the list:
    filtered_table = [d for d in table if list(d.keys())[0][2] == len(sentence)-1]
    
    #get the last dict of our BIO_list with the higher value
    current_dict = max(filtered_table, key=lambda d: list(d.values())[0]['value'])

    #insert in the BIO list
    BIO_list.insert(0, list(current_dict.keys())[0][1])
    final_table.insert(0, current_dict)    
    
    while list(current_dict.values())[0]['prev_key'] != ('<B>', '<B>', -1): #while not the first element
        current_dict = [d for d in table if list(current_dict.values())[0]['prev_key'] in d ][0]
        BIO_list.insert(0, list(current_dict.keys())[0][1] )
        final_table.insert(0, current_dict)
        
    if printIt=='result': 
        # Flatten dictionaries and convert to DataFrame
        import pandas as pd
        pd.set_option('display.max_columns', None)  # This option displays all columns
        pd.set_option('display.expand_frame_repr', False)  # This option prevents pandas from wrapping tables over multiple lines
        pd.set_option('display.max_rows', None)  # This option displays all rows
        rows = []
        for d in final_table:
            for key, value in d.items():
                # Flatten the keys and values
                flat_dict = {**{"key": key}, **value}
                rows.append(flat_dict)

        df = pd.DataFrame(rows)        
        print(df, "\n")
    
    return BIO_list

In [11]:
print(viterbi(hmm,["Is sad that she 's missing Cowboy Mouth tonight :(", ['O', 'O', 'O', 'O', 'O', 'O', 'B-musicartist', 'I-musicartist', 'O', 'O']]))
print(viterbi(hmm,['is up and ready for his last day on the punt until Stakes Day ... #sadbuttrue', ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-other', 'I-other', 'O', 'O']]))
print(viterbi(hmm,['@sfgiantsfan55 omg Todo Cabio has been my fav since like `07 i still listen to their CD all the time i want the new one . alejate de mi ...', ['O', 'O', 'B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]))

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-other', 'I-other', 'O', 'O']
['O', 'O', 'B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


The dev data in `data/dev` is in the following format:

    STOP	O
    WHAT	O
    YOU'RE	O
    DOING	O
    AND	O
    GO	O
    GET	O
    #ExpelledMovieToNumberOne	O
    ON	O
    ITUNES	B-other
    BECAUSE	O
    IT'S	O
    ONLY	O
    2ND	O
    !!	O
    @camerondallas	O
    Shs	O
    ...

It is organized with two whitespace-seperated columns as seen above.

Let's write code to read in the dev data and store it as list called `dev` in the same way that we read in the training data.

In [12]:
dev = [' '.join(i.split('\n')).replace('\t', ' ').split() for i in open('data/dev', 'r').read().split('\n\n')]
dev = [[" ".join(i[::2]),i[1::2]] for i in dev]

The following is a python function inspired by **"The conlleval script"** written in Perl. It is often used to evaluate the performance of Named Entity Recognition (NER) models. It is named after the Conference on Natural Language Learning (CoNLL), where it was initially used for shared task evaluations.The conlleval script written in Perl is often used to evaluate the performance of Named Entity Recognition (NER) models. It is named after the Conference on Natural Language Learning (CoNLL), where it was initially used for shared task evaluations.

Run the cell without modifying it:

In [13]:
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelBinarizer
from itertools import chain
import numpy as np

def conlleval(labels_predicted, labels_correct, labels_all):
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(labels_correct)))
    y_pred_combined = lb.transform(list(chain.from_iterable(labels_predicted)))

    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}

    num_sentences = len(labels_predicted)
    total_tokens = np.sum([len(s) for s in labels_predicted])
    num_correct_sentences = sum([all(np.array(pred) == np.array(true)) for pred, true in zip(labels_predicted, labels_correct)])
    correct_sentences_percentage = num_correct_sentences / num_sentences * 100
    total_correct_tokens = sum([sum(np.array(pred) == np.array(true)) for pred, true in zip(labels_predicted, labels_correct)])
    total_correct_tokens_percentage = total_correct_tokens / total_tokens * 100
    tag_counts = {tag: sum([s.count(tag) for s in labels_predicted]) for tag in tagset}

    classification_report_dict = classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
        output_dict=True,
        zero_division=1,
    )
   
    classification_report_dict.pop('macro avg', None)
    classification_report_dict.pop('weighted avg', None)
    classification_report_dict.pop('samples avg', None)
    classification_report_dict.pop('micro avg', None)

    total_precision = precision_score(y_true_combined, y_pred_combined, average='weighted', zero_division=1)
    total_recall = recall_score(y_true_combined, y_pred_combined, average='weighted', zero_division=1)
    total_f1 = f1_score(y_true_combined, y_pred_combined, average='weighted', zero_division=1)
    total_line = f"{'Total':<15s} {total_precision:<10.2f} {total_recall:<10.2f} {total_f1:<10.2f}"

    report_lines = [f"{k:<15s} {classification_report_dict[k]['precision']:<10.2f} {classification_report_dict[k]['recall']:<10.2f} {classification_report_dict[k]['f1-score']:<10.2f}" for k in classification_report_dict if isinstance(classification_report_dict[k], dict)]
    report_lines.insert(0, "\n")
    report_lines.insert(1, f"{'TAG':<15s} {'Precision':<10s} {'Recall':<10s} {'F1-score':<10s}\n")
    report_lines.insert(2, total_line)
    report_lines.insert(3, '-'*50 + '\n') 
    classification_report_str = "\n".join(report_lines)

    additional_info_str = ''
    additional_info_str += f'Total tokens: {total_tokens}\n'
    additional_info_str += f'Total correct tokens: {total_correct_tokens} ({total_correct_tokens_percentage:.2f}%)\n'
    additional_info_str += f'Processed sentences: {num_sentences}\n'
    additional_info_str += f'Completely correct sentences: {num_correct_sentences} ({correct_sentences_percentage:.2f}%)\n'

    return additional_info_str + classification_report_str


We can use the conlleval function to evaluate our work.

In [14]:
predictions =  [viterbi(hmm, sent) for sent in dev if len(sent[0])!=0]
correct =      [d[1] for d in dev if len(d[1])>0]

viterbiEvaluation = conlleval(predictions, correct, hmm.tags)
print(viterbiEvaluation)

Total tokens: 16261
Total correct tokens: 11491 (70.67%)
Processed sentences: 1000
Completely correct sentences: 281 (28.10%)


TAG             Precision  Recall     F1-score  

Total           0.92       0.71       0.79      
--------------------------------------------------

B-company       0.86       0.15       0.26      
I-company       1.00       0.00       0.00      
B-facility      0.19       0.08       0.11      
I-facility      0.19       0.15       0.17      
B-geo-loc       0.73       0.19       0.30      
I-geo-loc       0.39       0.17       0.23      
B-movie         0.00       0.00       0.00      
I-movie         0.00       0.00       0.00      
B-musicartist   0.00       0.00       0.00      
I-musicartist   0.05       0.11       0.07      
B-other         0.06       0.23       0.09      
I-other         0.02       0.59       0.04      
B-person        0.14       0.27       0.18      
I-person        0.12       0.41       0.18      
B-product       0.33       0.16    